In [106]:
import pandas as pd

In [107]:
df = pd.read_csv("MOA_processed.csv.gz", compression="gzip")

In [108]:
df

,pert_iname,SMILES,dopamine_receptor_agonist,phosphodiesterase_inhibitor,benzodiazepine_receptor_agonist,tachykinin_antagonist,glutamate_receptor_antagonist,acetylcholine_receptor_agonist,glycogen_synthase_kinase_inhibitor,alpha_mannosidase_inhibitor,...,urology,dental,genetics,critical_care,transplant,metabolism,otolaryngology,radiology,Standardized_SMILES,Standardized_InChI
0,(R)-(-)-apomorphine,CN1CCc2cccc-3c2[C@H]1Cc1ccc(O)c(O)c-31,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,C[NH+]1CCc2cccc3c2C1Cc1ccc([O-])c([O-])c1-3,InChI=1S/C17H17NO2/c1-18-8-7-10-3-2-4-12-15(10...
1,(R)-(-)-rolipram,COc1ccc(cc1OC1CCCC1)[C@@H]1CNC(=O)C1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,COc1ccc(C2CNC(=O)C2)cc1OC1CCCC1,InChI=1S/C16H21NO3/c1-19-14-7-6-11(12-9-16(18)...
2,(R)-baclofen,NC[C@H](CC(O)=O)c1ccc(Cl)cc1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[NH3+]CC(CC(=O)[O-])c1ccc(Cl)cc1,InChI=1S/C10H12ClNO2/c11-9-3-1-7(2-4-9)8(6-12)...
3,(S)-(+)-rolipram,COc1ccc(cc1OC1CCCC1)[C@H]1CNC(=O)C1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,COc1ccc(C2CNC(=O)C2)cc1OC1CCCC1,InChI=1S/C16H21NO3/c1-19-14-7-6-11(12-9-16(18)...
4,"[sar9,met(o2)11]-substance-p",CC(C)C[C@H](NC(=O)CN(C)C(=O)[C@H](Cc1ccccc1)NC...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,CC(C)CC(N=C(O)C[NH+](C)C(=O)C(Cc1ccccc1)N=C(O)...,InChI=1S/C64H100N18O15S/c1-38(2)34-46(57(89)74...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6772,zoxazolamine,Nc1nc2cc(Cl)ccc2o1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,N=c1[n-]c2cc(Cl)ccc2o1,InChI=1S/C7H4ClN2O/c8-4-1-2-6-5(3-4)10-7(9)11-...
6773,ZSET1446,"O=C1N=C2C=CC=CN2C11Cc2ccccc2C1 |c:4,6,t:2|",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,O=C1[NH+]=C2C=CC=CN2C12Cc1ccccc1C2,InChI=1S/C15H12N2O/c18-14-15(17-8-4-3-7-13(17)...
6774,ZSTK-474,FC(F)c1nc2ccccc2n1-c1nc(nc(n1)N1CCOCC1)N1CCOCC1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,FC(F)c1nc2ccccc2n1-c1nc(N2CCOCC2)nc(N2CCOCC2)n1,InChI=1S/C19H21F2N7O2/c20-15(21)16-22-13-3-1-2...
6775,zuclopenthixol,OCCN1CCN(CC\C=C2\c3ccccc3Sc3ccc(Cl)cc23)CC1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,OCC[NH+]1CC[NH+](CCC=C2c3ccccc3Sc3ccc(Cl)cc32)CC1,InChI=1S/C22H25ClN2OS/c23-17-7-8-22-20(16-17)1...


In [109]:
import os
import sys
sys.path.append('/home/ss2686/03_DICTrank')

from scripts.stratified_split_helper import stratified_data_split

import pandas as pd
import pickle

# Loading the dictionary

datasets = {}

directory='../'
# Load datasets from given directory
for foldername in os.listdir(directory):
        
    if not foldername.startswith('.'):  # Ignore folders starting with a dot
            
        print(foldername)
        file_path = os.path.join(directory, foldername, f"{foldername}_processed.csv.gz")

        if os.path.exists(file_path):
            datasets[foldername] = pd.read_csv(file_path, compression='gzip')
        else:
            print(f"No matching file found for folder: {foldername}")

GeneExpressionSMILES
sider
MOA
CellPaintingSMILES
Cmax
DICTrank


In [110]:
smiles_list=[]
for featuresets in ["sider", "MOA", "Cmax", "DICTrank"]: 
    
    smiles_list.extend(datasets[featuresets].Standardized_SMILES.to_list())
    print(len(smiles_list))
    
smiles_list = list(set(smiles_list))

print(len(smiles_list))

1378
8155
8913
10148
6947


In [111]:
l = pd.DataFrame(smiles_list, columns=["Standardized_SMILES"])
l

,Standardized_SMILES
0,NC(=[NH2+])NC(=O)Cc1c(Cl)cccc1Cl
1,N=c1[n-]c(=O)c2c(CCc3ccc(C(=O)NC(CCC(=O)[O-])C...
2,CCOC(=O)C(CCc1ccccc1)[NH2+]C(C)C(=O)[NH+](CC(=...
3,CC1CCC2(C(=O)[O-])CCC3(C)C(=CCC4C5(C)CC(O)C(O)...
4,COCCN(C(=O)c1cc(OC)c(OC)c(OC)c1)c1nc(-c2ccccc2...
...,...
6942,CC1Cc2ccccc2N1[N-]C(=O)c1ccc(Cl)c(S(N)(=O)=O)c1
6943,Cc1c(C(=O)NC2CCCCC2)sc2nc3ccc(N)cc3n12
6944,CCC(C)CCCCC(=O)NC(CC[NH2+]CS(=O)(=O)[O-])C(=O)...
6945,[NH3+]C1CCCN(c2c(C=C3SC(=O)[N-]C3=O)cccc2-c2cc...


In [112]:
df = df[df["Standardized_SMILES"].isin(smiles_list)]

In [113]:
df = df.drop(columns=['pert_iname', 'SMILES'])

# Get a list of the columns
cols = df.columns.tolist()

# Pop the 'SMILES' column and insert it at the second position
cols.insert(0, cols.pop(cols.index('Standardized_SMILES')))
cols.insert(1, cols.pop(cols.index('Standardized_InChI')))

# Reorder the DataFrame columns
df = df[cols]
df

,Standardized_SMILES,Standardized_InChI,dopamine_receptor_agonist,phosphodiesterase_inhibitor,benzodiazepine_receptor_agonist,tachykinin_antagonist,glutamate_receptor_antagonist,acetylcholine_receptor_agonist,glycogen_synthase_kinase_inhibitor,alpha_mannosidase_inhibitor,...,allergy,orthopedics,urology,dental,genetics,critical_care,transplant,metabolism,otolaryngology,radiology
0,C[NH+]1CCc2cccc3c2C1Cc1ccc([O-])c([O-])c1-3,InChI=1S/C17H17NO2/c1-18-8-7-10-3-2-4-12-15(10...,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,COc1ccc(C2CNC(=O)C2)cc1OC1CCCC1,InChI=1S/C16H21NO3/c1-19-14-7-6-11(12-9-16(18)...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[NH3+]CC(CC(=O)[O-])c1ccc(Cl)cc1,InChI=1S/C10H12ClNO2/c11-9-3-1-7(2-4-9)8(6-12)...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,COc1ccc(C2CNC(=O)C2)cc1OC1CCCC1,InChI=1S/C16H21NO3/c1-19-14-7-6-11(12-9-16(18)...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CC(C)CC(N=C(O)C[NH+](C)C(=O)C(Cc1ccccc1)N=C(O)...,InChI=1S/C64H100N18O15S/c1-38(2)34-46(57(89)74...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6772,N=c1[n-]c2cc(Cl)ccc2o1,InChI=1S/C7H4ClN2O/c8-4-1-2-6-5(3-4)10-7(9)11-...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6773,O=C1[NH+]=C2C=CC=CN2C12Cc1ccccc1C2,InChI=1S/C15H12N2O/c18-14-15(17-8-4-3-7-13(17)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6774,FC(F)c1nc2ccccc2n1-c1nc(N2CCOCC2)nc(N2CCOCC2)n1,InChI=1S/C19H21F2N7O2/c20-15(21)16-22-13-3-1-2...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6775,OCC[NH+]1CC[NH+](CCC=C2c3ccccc3Sc3ccc(Cl)cc32)CC1,InChI=1S/C22H25ClN2OS/c23-17-7-8-22-20(16-17)1...,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [114]:
from sklearn.feature_selection import VarianceThreshold

# Specify a variance threshold. Features with a variance below this threshold will be removed.
# As an example, we'll use a threshold of 0.2 (you can adjust this value based on your needs).
selector = VarianceThreshold(threshold=0.001)

# Considering only the subset of columns specified
subset = df[df.columns[2:]]

# Fit the variance threshold to the data
selector.fit(subset)

# Transform the data
subset_transformed = selector.transform(subset)

# Convert the transformed data back to a dataframe
df_transformed = pd.DataFrame(subset_transformed, columns=subset.columns[selector.get_support()])
df_transformed

# If you want to keep the original first two and last two columns and just replace the columns between them with the transformed data
df = pd.concat([df[df.columns[:2]], df_transformed], axis=1)
df

,Standardized_SMILES,Standardized_InChI,dopamine_receptor_agonist,phosphodiesterase_inhibitor,benzodiazepine_receptor_agonist,tachykinin_antagonist,glutamate_receptor_antagonist,acetylcholine_receptor_agonist,glycogen_synthase_kinase_inhibitor,potassium_channel_activator,...,gastroenterology,hematology,allergy,orthopedics,urology,dental,critical_care,metabolism,otolaryngology,radiology
0,C[NH+]1CCc2cccc3c2C1Cc1ccc([O-])c([O-])c1-3,InChI=1S/C17H17NO2/c1-18-8-7-10-3-2-4-12-15(10...,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,COc1ccc(C2CNC(=O)C2)cc1OC1CCCC1,InChI=1S/C16H21NO3/c1-19-14-7-6-11(12-9-16(18)...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[NH3+]CC(CC(=O)[O-])c1ccc(Cl)cc1,InChI=1S/C10H12ClNO2/c11-9-3-1-7(2-4-9)8(6-12)...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,COc1ccc(C2CNC(=O)C2)cc1OC1CCCC1,InChI=1S/C16H21NO3/c1-19-14-7-6-11(12-9-16(18)...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CC(C)CC(N=C(O)C[NH+](C)C(=O)C(Cc1ccccc1)N=C(O)...,InChI=1S/C64H100N18O15S/c1-38(2)34-46(57(89)74...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6772,N=c1[n-]c2cc(Cl)ccc2o1,InChI=1S/C7H4ClN2O/c8-4-1-2-6-5(3-4)10-7(9)11-...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6773,O=C1[NH+]=C2C=CC=CN2C12Cc1ccccc1C2,InChI=1S/C15H12N2O/c18-14-15(17-8-4-3-7-13(17)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6774,FC(F)c1nc2ccccc2n1-c1nc(N2CCOCC2)nc(N2CCOCC2)n1,InChI=1S/C19H21F2N7O2/c20-15(21)16-22-13-3-1-2...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6775,OCC[NH+]1CC[NH+](CCC=C2c3ccccc3Sc3ccc(Cl)cc32)CC1,InChI=1S/C22H25ClN2OS/c23-17-7-8-22-20(16-17)1...,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [115]:
df

,Standardized_SMILES,Standardized_InChI,dopamine_receptor_agonist,phosphodiesterase_inhibitor,benzodiazepine_receptor_agonist,tachykinin_antagonist,glutamate_receptor_antagonist,acetylcholine_receptor_agonist,glycogen_synthase_kinase_inhibitor,potassium_channel_activator,...,gastroenterology,hematology,allergy,orthopedics,urology,dental,critical_care,metabolism,otolaryngology,radiology
0,C[NH+]1CCc2cccc3c2C1Cc1ccc([O-])c([O-])c1-3,InChI=1S/C17H17NO2/c1-18-8-7-10-3-2-4-12-15(10...,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,COc1ccc(C2CNC(=O)C2)cc1OC1CCCC1,InChI=1S/C16H21NO3/c1-19-14-7-6-11(12-9-16(18)...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[NH3+]CC(CC(=O)[O-])c1ccc(Cl)cc1,InChI=1S/C10H12ClNO2/c11-9-3-1-7(2-4-9)8(6-12)...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,COc1ccc(C2CNC(=O)C2)cc1OC1CCCC1,InChI=1S/C16H21NO3/c1-19-14-7-6-11(12-9-16(18)...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CC(C)CC(N=C(O)C[NH+](C)C(=O)C(Cc1ccccc1)N=C(O)...,InChI=1S/C64H100N18O15S/c1-38(2)34-46(57(89)74...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6772,N=c1[n-]c2cc(Cl)ccc2o1,InChI=1S/C7H4ClN2O/c8-4-1-2-6-5(3-4)10-7(9)11-...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6773,O=C1[NH+]=C2C=CC=CN2C12Cc1ccccc1C2,InChI=1S/C15H12N2O/c18-14-15(17-8-4-3-7-13(17)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6774,FC(F)c1nc2ccccc2n1-c1nc(N2CCOCC2)nc(N2CCOCC2)n1,InChI=1S/C19H21F2N7O2/c20-15(21)16-22-13-3-1-2...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6775,OCC[NH+]1CC[NH+](CCC=C2c3ccccc3Sc3ccc(Cl)cc32)CC1,InChI=1S/C22H25ClN2OS/c23-17-7-8-22-20(16-17)1...,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [116]:
df.Standardized_SMILES.nunique()

6613

In [119]:
# Step 1 & 2: Group by 'Standardized_SMILES' and take the median
grouped_df = df.groupby('Standardized_SMILES').mean().reset_index()

# Step 3: Replace anything greater than 0 as 1 for the specific columns 
# (excluding 'Standardized_SMILES' and 'Standardized_InChI' columns)
columns_to_modify = grouped_df.columns.difference(['Standardized_SMILES', 'Standardized_InChI'])
for col in columns_to_modify:
    grouped_df[col] = grouped_df[col].apply(lambda x: 1 if x > 0 else 0)

grouped_df

,Standardized_SMILES,dopamine_receptor_agonist,phosphodiesterase_inhibitor,benzodiazepine_receptor_agonist,tachykinin_antagonist,glutamate_receptor_antagonist,acetylcholine_receptor_agonist,glycogen_synthase_kinase_inhibitor,potassium_channel_activator,SRC_inhibitor,...,gastroenterology,hematology,allergy,orthopedics,urology,dental,critical_care,metabolism,otolaryngology,radiology
0,BrC1C(Br)C(Br)C(Br)C(Br)C1Br,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Brc1c(Br)c(Br)c2[n-]nnc2c1Br,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Brc1c(NC2=[NH+]CCN2)ccc2nccnc12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Brc1cc2c(cc1C1[NH2+]c3ccccc3C3C=CCC31)OCO2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Brc1ccc([NH2+]C2C3CC4CC(C3)CC2C4)cc1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,c1cncc(C[NH+]2CCC(n3ncc4c(N5CCOCC5)nc(-c5ccc6[...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6609,c1coc(-c2nnc3sc(C4CCCCC4)nn23)c1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6610,c1coc(CN=c2[n-]cnc3[n-]cnc23)c1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6611,c1nc(CC2CC[NH2+]CC2)c[n-]1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
grouped_df.to_csv("../../processed_feature_selected/MOA/MOA_processed_feature_selected.csv.gz", compression="gzip", index=False)

In [1]:
import pandas as pd
df = pd.read_csv("../../processed_feature_selected/MOA/MOA_processed_feature_selected.csv.gz", compression="gzip")

In [2]:
df

,Standardized_SMILES,dopamine_receptor_agonist,phosphodiesterase_inhibitor,benzodiazepine_receptor_agonist,tachykinin_antagonist,glutamate_receptor_antagonist,acetylcholine_receptor_agonist,glycogen_synthase_kinase_inhibitor,potassium_channel_activator,SRC_inhibitor,...,gastroenterology,hematology,allergy,orthopedics,urology,dental,critical_care,metabolism,otolaryngology,radiology
0,BrC1C(Br)C(Br)C(Br)C(Br)C1Br,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Brc1c(Br)c(Br)c2[n-]nnc2c1Br,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Brc1c(NC2=[NH+]CCN2)ccc2nccnc12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Brc1cc2c(cc1C1[NH2+]c3ccccc3C3C=CCC31)OCO2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Brc1ccc([NH2+]C2C3CC4CC(C3)CC2C4)cc1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,c1cncc(C[NH+]2CCC(n3ncc4c(N5CCOCC5)nc(-c5ccc6[...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6609,c1coc(-c2nnc3sc(C4CCCCC4)nn23)c1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6610,c1coc(CN=c2[n-]cnc3[n-]cnc23)c1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6611,c1nc(CC2CC[NH2+]CC2)c[n-]1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
moa_cols = df.columns.tolist()[1:265]

In [36]:
MOA_processed_feature_selected_moa_cols = df[["Standardized_SMILES"]+moa_cols]
MOA_processed_feature_selected_moa_cols

,Standardized_SMILES,dopamine_receptor_agonist,phosphodiesterase_inhibitor,benzodiazepine_receptor_agonist,tachykinin_antagonist,glutamate_receptor_antagonist,acetylcholine_receptor_agonist,glycogen_synthase_kinase_inhibitor,potassium_channel_activator,SRC_inhibitor,...,vitamin_K,T-type_calcium_channel_blocker,serotonin_reuptake_inhibitor,tyrosine_kinase_receptor_inhibitor,PABA_antagonist,AP_inhibitor,phosphorylase_inhibitor,peptidase_inhibitor,like_(NRF2)_activator,phosphatase_inhibitor
0,BrC1C(Br)C(Br)C(Br)C(Br)C1Br,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Brc1c(Br)c(Br)c2[n-]nnc2c1Br,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Brc1c(NC2=[NH+]CCN2)ccc2nccnc12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Brc1cc2c(cc1C1[NH2+]c3ccccc3C3C=CCC31)OCO2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Brc1ccc([NH2+]C2C3CC4CC(C3)CC2C4)cc1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,c1cncc(C[NH+]2CCC(n3ncc4c(N5CCOCC5)nc(-c5ccc6[...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6609,c1coc(-c2nnc3sc(C4CCCCC4)nn23)c1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6610,c1coc(CN=c2[n-]cnc3[n-]cnc23)c1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6611,c1nc(CC2CC[NH2+]CC2)c[n-]1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
MOA_processed_feature_selected_moa_cols.to_csv("../../processed_feature_selected/MOA/MOA_processed_feature_selected_moa_cols.csv.gz", compression="gzip", index=False)

In [32]:
disease_area_cols = df.columns.tolist()[265:415]

In [31]:
target_cols = df.columns.tolist()[415:866]

In [30]:
indication_cols= df.columns.tolist()[866:]